## Move joint for testing initialization

In [2]:
import sys
import time
from MachineMotion import *
from tkinter import *
from tkinter import ttk
import numpy as np
import requests
from matplotlib import pyplot as plt
from PIL import ImageTk, Image
import io
import cv2
import time
import rtde_receive
import rtde_control
import requests
from matplotlib import pyplot as plt
from PIL import ImageTk, Image
from robotiq_preamble import ROBOTIQ_PREAMBLE
import time

#####################################
### Gripper control
#####################################
class RobotiqGripper(object):
    def __init__(self, rtde_c): 
        
        self.rtde_c = rtde_c

    def call(self, script_name, script_function):
        return self.rtde_c.sendCustomScriptFunction(
            "ROBOTIQ_" + script_name,
            ROBOTIQ_PREAMBLE + script_function)

    def activate(self):
        ret = self.call("ACTIVATE", "rq_activate()")
        time.sleep(5)  # HACK
        return ret

    def set_speed(self, speed): # speed (int): speed as a percentage [0-100]
        return self.call("SET_SPEED", "rq_set_speed_norm(" + str(speed) + ")")

    def set_force(self, force):# force (int): force as a percentage [0-100]
        return self.call("SET_FORCE", "rq_set_force_norm(" + str(force) + ")")

    def move(self, pos_in_mm):
        return self.call("MOVE", "rq_move_and_wait_mm(" + str(pos_in_mm) + ")")

    def open(self):
        return self.call("OPEN", "rq_open_and_wait()")

    def close(self):
        return self.call("CLOSE", "rq_close_and_wait()")

#####################################
### Conveyor control
#####################################


class ConveryorControl:
    def __init__(self, ip):
        self.controller = MachineMotion(machineIp=ip)
        self.speed = 50
        self.acceleration = 50
        self.controller.setSpeed(self.speed)
        self.controller.setAcceleration(self.acceleration)

    def set_acceleration(self, acceleration, axis=1):
        self.acceleration = acceleration
        self.controller.setAcceleration(acceleration)
    
    def set_velocity(self, velocity, axis=1):
        self.speed = velocity
        self.controller.setSpeed(velocity)

    def move_continuous(self, direction, axis=1):
        self.controller.moveContinuous(axis, self.speed * direction, self.acceleration)
    
    def stop_continuous(self, axis=1):
        self.controller.stopMoveContinuous(axis, self.acceleration)
        
    def sensor(self, axis=1):
        self.sensor_value = self.controller.digitalRead(1,3) #(networkid,pin)
        return self.sensor_value
        
#####################################
### Variable
#####################################
how_long = 180 #sec

host = "192.168.10.83"
TCP_conveyor_before_grip = [48.82, 3.86, -129.20, 5.21, 57.62, -222.12]
TCP_conveyor_2x2_grip = [53.72, -1.08, -124.08, 5.72, 54.47, -222.13]
TCP_conveyor_1x4_grip = [50.32, -2.73, -124.76, 7.72, 56.87, -215.82]
TCP_conveyor_1x3_grip = [50.32, -2.73, -124.76, 7.72, 56.87, -215.82]

TCP_plate_red_1x3_grip = [121.80, -17.03, 44.30, -83.75, 53.95, -39.95]
TCP_plate_red_1x3_grip2 = [116.78, -13.00, 56.07, -104.73, 53.86, -39.84]
TCP_plate_green_1x4_grip = [115.70, -32.98, 66.06, -100.65, 54.16, -30.07]
TCP_plate_blue_1x4_grip = [102.13, -51.38, 83.33, -112.62, 44.89, -12.56]

TCP_plate_2x2_grip1 = [117.93, -60.69, 102.88, -104.89, 51.40, 50.12] # drop
TCP_plate_2x2_grip1_1 = [110.74, -54.77, 115.02, -126.43, 50.65, 50.42]
TCP_plate_2x2_grip2 = [109.42, -60.13, 118.84, -108.74, 50.85, 51.17] # gripper close + up
TCP_plate_2x2_grip3 = [123.71, -48.32, 96.10, -106.46, 56.95, 41.49] # left side
TCP_plate_2x2_grip4 = [121.31, -55.97, 101.27, -109.92, 56.46, 47.80] # from left side -> push
TCP_plate_2x2_grip5 = [124.01, -70.08, 120.39, -112.52, 52.95, 132.66] # upper side
TCP_plate_2x2_grip6 = [120.84, -68.11, 109.88, -98.35, 56.83, 133.60] # upper side
##############################################################################
TCP_plate_2x2_grip7 = [116.56, -61.10, 106.75, -108.94, 53.14, 51.86] # stacking
# gripper open
TCP_plate_2x2_grip8 = [109.48, -61.79, 119.18, -115.81, 51.33, 57.75]
# gripper close
TCP_plate_2x2_grip9 = [112.76, -61.84, 111.07, -113.62, 48.14, 55.25]
TCP_plate_2x2_grip10 = [113.50, -62.00, 108.63, -112.29, 48.05, 55.27]


TCP_conveyor_camera = [31.99, 16.24, -92.94, -104.63, 64.78, -183.90] # conveyor
TCP_starting_point = [90, -45, 90, -135, 45, 0]


rtde_c = rtde_control.RTDEControlInterface(host)
rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_1x3_grip])
#rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip1])
rtde_c.disconnect()
"""
gripper = RobotiqGripper(rtde_c)
gripper.activate()
gripper.set_speed(100)
gripper.set_force(100)
gripper.open()
gripper.close()
"""

C:\Users\jeon\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jeon\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\jeon\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


'\ngripper = RobotiqGripper(rtde_c)\ngripper.activate()\ngripper.set_speed(100)\ngripper.set_force(100)\ngripper.open()\ngripper.close()\n'

In [1]:
rtde_c.disconnect()

NameError: name 'rtde_c' is not defined

In [ ]:
gripper = RobotiqGripper(rtde_c)
gripper.activate()
gripper.close()

## Main code

In [2]:
import sys
import time
from MachineMotion import *
from tkinter import *
from tkinter import ttk
import numpy as np
import requests
from matplotlib import pyplot as plt
from PIL import ImageTk, Image
import io
import cv2
import time
import rtde_receive
import rtde_control
import requests
from matplotlib import pyplot as plt
from PIL import ImageTk, Image
from robotiq_preamble import ROBOTIQ_PREAMBLE
import time

#####################################
### Gripper control
#####################################
class RobotiqGripper(object):
    def __init__(self, rtde_c): 
        
        self.rtde_c = rtde_c

    def call(self, script_name, script_function):
        return self.rtde_c.sendCustomScriptFunction(
            "ROBOTIQ_" + script_name,
            ROBOTIQ_PREAMBLE + script_function)

    def activate(self):
        ret = self.call("ACTIVATE", "rq_activate()")
        time.sleep(5)  # HACK
        return ret

    def set_speed(self, speed): # speed (int): speed as a percentage [0-100]
        return self.call("SET_SPEED", "rq_set_speed_norm(" + str(speed) + ")")

    def set_force(self, force):# force (int): force as a percentage [0-100]
        return self.call("SET_FORCE", "rq_set_force_norm(" + str(force) + ")")

    def move(self, pos_in_mm):
        return self.call("MOVE", "rq_move_and_wait_mm(" + str(pos_in_mm) + ")")

    def open(self):
        return self.call("OPEN", "rq_open_and_wait()")

    def close(self):
        return self.call("CLOSE", "rq_close_and_wait()")

#####################################
### Conveyor control
#####################################


class ConveryorControl:
    def __init__(self, ip):
        self.controller = MachineMotion(machineIp=ip)
        self.speed = 50
        self.acceleration = 50
        self.controller.setSpeed(self.speed)
        self.controller.setAcceleration(self.acceleration)

    def set_acceleration(self, acceleration, axis=1):
        self.acceleration = acceleration
        self.controller.setAcceleration(acceleration)
    
    def set_velocity(self, velocity, axis=1):
        self.speed = velocity
        self.controller.setSpeed(velocity)

    def move_continuous(self, direction, axis=1):
        self.controller.moveContinuous(axis, self.speed * direction, self.acceleration)
    
    def stop_continuous(self, axis=1):
        self.controller.stopMoveContinuous(axis, self.acceleration)
        
    def sensor(self, axis=1):
        self.sensor_value = self.controller.digitalRead(1,3) #(networkid,pin)
        return self.sensor_value
        
#####################################
### Variable
#####################################
how_long = 180 #sec

host = "192.168.10.83"

TCP_starting_point = [90, -45, 90, -135, 45, 0]
TCP_conveyor_camera = [32.77, 16.90, -92.62, -99.59, 64.79, -183.88]
TCP_conveyor_camera = [31.99, 16.24, -92.94, -104.63, 64.78, -183.90] # conveyor
#TCP_conveyor_grip_1 = [50.12, 8.68, -128.19, -6.16, 58.11, -221.78]
#TCP_conveyor_grip_2 = [53.07, -2.07, -126.47, 8.98, 55.37, -221.50]

TCP_conveyor_before_grip = [48.82, 3.86, -129.20, 5.21, 57.62, -222.12]
TCP_conveyor_2x2_grip = [53.72, -1.08, -124.08, 5.72, 54.47, -222.13]
TCP_conveyor_1x4_grip = [50.32, -2.73, -124.76, 7.72, 56.87, -215.82]
TCP_conveyor_1x3_grip = [50.32, -2.73, -124.76, 7.72, 56.87, -215.82]

TCP_plate_red_1x3_grip = [121.80, -17.03, 44.30, -83.75, 53.95, -39.95]
TCP_plate_red_1x3_grip2 = [116.78, -13.00, 56.07, -104.73, 53.86, -39.84]
TCP_plate_green_1x4_grip = [115.70, -32.98, 66.06, -100.65, 54.16, -30.07]
TCP_plate_blue_1x4_grip = [102.13, -51.38, 83.33, -112.62, 44.89, -12.56]

TCP_plate_2x2_grip1 = [117.93, -60.69, 102.88, -104.89, 51.40, 50.12] # drop
TCP_plate_2x2_grip1_1 = [110.74, -54.77, 115.02, -126.43, 50.65, 50.42]
TCP_plate_2x2_grip2 = [109.42, -60.13, 118.84, -108.74, 50.85, 51.17] # gripper close + up
TCP_plate_2x2_grip3 = [123.71, -48.32, 96.10, -106.46, 56.95, 41.49] # left side
TCP_plate_2x2_grip4 = [121.31, -55.97, 101.27, -109.92, 56.46, 47.80] # from left side -> push
TCP_plate_2x2_grip5 = [124.01, -70.08, 120.39, -112.52, 52.95, 132.66] # upper side
TCP_plate_2x2_grip6 = [120.84, -68.11, 109.88, -98.35, 56.83, 133.60] # upper side
##############################################################################
TCP_plate_2x2_grip7 = [116.56, -61.10, 106.75, -108.94, 53.14, 51.86] # stacking
# gripper open
TCP_plate_2x2_grip8 = [109.48, -61.79, 119.18, -115.81, 51.33, 57.75]
# gripper close
TCP_plate_2x2_grip9 = [112.76, -61.84, 111.07, -113.62, 48.14, 55.25]
TCP_plate_2x2_grip10 = [113.50, -62.00, 108.63, -112.29, 48.05, 55.27]


rtde_c = rtde_control.RTDEControlInterface(host)

time.sleep(1)
gripper = RobotiqGripper(rtde_c)
gripper.activate()
gripper.set_speed(100)
gripper.set_force(100)
gripper.open()
time.sleep(3)
#
# rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
# 

#####################################
### Activate function
#####################################
start_time = time.time()

aa = 1
cc = 1
status = 0
x22 = 0

cc1 = ConveryorControl("192.168.10.122")
cc1.set_acceleration(50)
cc1.set_velocity(50)
cc1.move_continuous(1)

while (aa != 0):
    
    ###########
    ### Move conveyor
    ###########
    while (cc != 0):
        cc1.move_continuous(1)
        temp = []
        
        if cc1.sensor() == 1: # detect object
            cc1.stop_continuous()
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
            time.sleep(2)
            bb = 1 # Activate camera
            break
    
    ##########
    ### Camera detect 
    ##########
    
    while (bb != 0):
        
        img = np.zeros((480,640,3))
        resp = requests.get("http://"+ "192.168.10.83"+":4242/current.jpg?type=color", stream=True).raw
        img = np.asarray(bytearray(resp.read()), dtype="uint8")
        img = cv2.imdecode(img, cv2.IMREAD_COLOR)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
        lb_red = np.array([150, 50, 70], "uint8")
        ub_red = np.array([200, 255, 255], "uint8")
        red_mask = cv2.inRange(hsv, lb_red, ub_red)

        lb_green = np.array([40, 150, 50], "uint8")
        ub_green = np.array([70, 255, 100], "uint8")
        green_mask = cv2.inRange(hsv, lb_green, ub_green)

        lb_blue = np.array([100, 200, 50], "uint8")
        ub_blue = np.array([130, 255, 255], "uint8")
        blue_mask = cv2.inRange(hsv, lb_blue, ub_blue)

        lb_yellow = np.array([0, 100, 100], "uint8")
        ub_yellow = np.array([50, 255, 255], "uint8")
        yellow_mask = cv2.inRange(hsv, lb_yellow, ub_yellow)

        kernel = np.ones((5, 5), "uint8")

        red_mask = cv2.dilate(red_mask, kernel)
        green_mask = cv2.dilate(green_mask, kernel)
        blue_mask = cv2.dilate(blue_mask, kernel)
        yellow_mask = cv2.dilate(yellow_mask, kernel)

        ########################
        ### RGB/Area of interests
        ########################
        min_area_red = 5000
        max_area_red = 20000

        min_area_green =  5000
        max_area_green =  20000

        min_area_blue =  3000
        max_area_blue =   20000

        min_area_yellow =  5000
        max_area_yellow =   20000

        area_x_min = 200
        area_x_max = 350
        area_y_min = 150
        area_y_max = 500

        ########################
        ### Red
        ########################
        contours, hierarchy = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if (min_area_red <= area and max_area_red >= area):
                x,y,w,h = cv2.boundingRect(contour)
                if (area_x_min <= x and x <= area_x_max):
                    if (area_y_min <= y and y <= area_y_max):
                        if w <= 80:
                            temp.append("red_1x3")
                        if w > 95:
                            temp.append("red_2x2")
                            

        ########################
        ### Green
        ########################
        contours, hierarchy = cv2.findContours(green_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if (min_area_green <= area and max_area_green >= area):
                x,y,w,h = cv2.boundingRect(contour)
                if (area_x_min <= x and x <= area_x_max):
                    if (area_y_min <= y and y <= area_y_max):
                        if h <= 180:
                            temp.append("green_1x2")
                        if h > 180:
                            temp.append("green_1x4")

        ########################
        ### Blue
        ########################
        contours, hierarchy = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if (min_area_blue <= area and max_area_blue >= area):
                x,y,w,h = cv2.boundingRect(contour)
                if (area_x_min <= x and x <= area_x_max):
                    if (area_y_min <= y and y <= area_y_max):
                        if h <= 100:
                            temp.append("blue_1x1")
                        if h > 100:
                            temp.append("blue_1x4")

        ########################
        ### Yellow
        ########################
        contours, hierarchy = cv2.findContours(yellow_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if area >= min_area_yellow and area <= max_area_yellow:
                x,y,w,h = cv2.boundingRect(contour)
                if (area_x_min <= x and x <= area_x_max):
                    if (area_y_min <= y and y <= area_y_max):
                        temp.append("yellow_2x2")
        
        if len(temp) >= 10:
            temp_color, temp_number = np.unique(np.array(temp), return_counts = True)
            if np.max(temp_number) >= 5:
                status = temp_color[np.argmax(temp_number)]
                bb = 0
    ###########
    ### Stacking
    ###########
    
    if status in ["red_2x2", "yellow_2x2"]:
        if x22 == 0:
            print(status)

            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip])
            time.sleep(0.5)
            gripper.close()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip1])
            time.sleep(0.5)
            gripper.open()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip1_1])
            time.sleep(0.5)
            gripper.close()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip2])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip3])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip4])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip3])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip5])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip6])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip5])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
            time.sleep(0.5)
            gripper.open()
            time.sleep(1)
            x22 = 1
            status = 0
            cc = 1
            
    if status in ["red_2x2", "yellow_2x2"]:      
        if x22 == 1:
            print(status)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip])
            time.sleep(0.5)
            gripper.close()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip7])
            gripper.open()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip8])
            time.sleep(0.5)
            gripper.close()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip9])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip10])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip9])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
            time.sleep(0.5)
            gripper.open()
            time.sleep(1)
            status = 0
            cc = 1
    
    if status == "red_1x3":
        print(status)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_1x3_grip])
        time.sleep(0.5)
        gripper.close()
        time.sleep(1)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_red_1x3_grip])
        gripper.open()
        time.sleep(1)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_red_1x3_grip2])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
        time.sleep(0.5)
        status = 0
        cc = 1
     
    if status == "green_1x4":
        print(status)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_1x4_grip])
        time.sleep(0.5)
        gripper.close()
        time.sleep(1)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_green_1x4_grip])
        gripper.open()
        time.sleep(1)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
        time.sleep(0.5)
        status = 0
        cc = 1
        
    if status == "blue_1x4":
        print(status)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_1x4_grip])
        time.sleep(0.5)
        gripper.close()
        time.sleep(1)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_blue_1x4_grip])
        gripper.open()
        time.sleep(1)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
        time.sleep(0.5)
        status = 0
        cc = 1

C:\Users\jeon\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jeon\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\jeon\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


KeyboardInterrupt: 

In [2]:
rtde_c.disconnect()

In [3]:
import sys
import time
from MachineMotion import *
from tkinter import *
from tkinter import ttk
import numpy as np
import requests
from matplotlib import pyplot as plt
from PIL import ImageTk, Image
import io
import cv2
import time
import rtde_receive
import rtde_control
import requests
from matplotlib import pyplot as plt
from PIL import ImageTk, Image
from robotiq_preamble import ROBOTIQ_PREAMBLE
import time

#####################################
### Gripper control
#####################################
class RobotiqGripper(object):
    def __init__(self, rtde_c): 
        
        self.rtde_c = rtde_c

    def call(self, script_name, script_function):
        return self.rtde_c.sendCustomScriptFunction(
            "ROBOTIQ_" + script_name,
            ROBOTIQ_PREAMBLE + script_function)

    def activate(self):
        ret = self.call("ACTIVATE", "rq_activate()")
        time.sleep(5)  # HACK
        return ret

    def set_speed(self, speed): # speed (int): speed as a percentage [0-100]
        return self.call("SET_SPEED", "rq_set_speed_norm(" + str(speed) + ")")

    def set_force(self, force):# force (int): force as a percentage [0-100]
        return self.call("SET_FORCE", "rq_set_force_norm(" + str(force) + ")")

    def move(self, pos_in_mm):
        return self.call("MOVE", "rq_move_and_wait_mm(" + str(pos_in_mm) + ")")

    def open(self):
        return self.call("OPEN", "rq_open_and_wait()")

    def close(self):
        return self.call("CLOSE", "rq_close_and_wait()")

#####################################
### Conveyor control
#####################################


class ConveryorControl:
    def __init__(self, ip):
        self.controller = MachineMotion(machineIp=ip)
        self.speed = 50
        self.acceleration = 50
        self.controller.setSpeed(self.speed)
        self.controller.setAcceleration(self.acceleration)

    def set_acceleration(self, acceleration, axis=1):
        self.acceleration = acceleration
        self.controller.setAcceleration(acceleration)
    
    def set_velocity(self, velocity, axis=1):
        self.speed = velocity
        self.controller.setSpeed(velocity)

    def move_continuous(self, direction, axis=1):
        self.controller.moveContinuous(axis, self.speed * direction, self.acceleration)
    
    def stop_continuous(self, axis=1):
        self.controller.stopMoveContinuous(axis, self.acceleration)
        
    def sensor(self, axis=1):
        self.sensor_value = self.controller.digitalRead(1,3) #(networkid,pin)
        return self.sensor_value
        
#####################################
### Variable
#####################################
how_long = 180 #sec

host = "192.168.10.83"

TCP_starting_point = [90, -45, 90, -135, 45, 0]

TCP_conveyor_camera = [31.99, 16.24, -92.94, -104.63, 64.78, -183.90] # conveyor

# yellow 2x2
# 54.46, 6.01, -134.60, 10.21, 55.85, -226.14
# 56.20, -0.80, -130.15, 10.67, 50.83, -226.06

# red 2x2
# 54.57, 10.49, -136.08, 4.33, 56.23, -224.01
# 56.97, 0.72, -131.40, 12.13, 51.11, -227.56

TCP_conveyor_before_grip = [41.88, 27.52, -146.00, -2.96, 64.10, -227.15]
TCP_conveyor_2x2_grip_red_1 = [54.57, 10.49, -136.08, 4.33, 56.23, -224.01]
TCP_conveyor_2x2_grip_red_2 = [56.97, 0.72, -131.40, 12.13, 51.11, -227.56]

TCP_conveyor_2x2_grip_yellow_1 = [54.46, 6.01, -134.60, 10.21, 55.85, -226.14]
TCP_conveyor_2x2_grip_yellow_2 = [56.20, -0.80, -130.15, 10.67, 50.83, -226.06]

TCP_conveyor_1x_grip = [53.17, 2.07, -133.24, 10.33, 53.03, -227.04]

TCP_plate_red_1x3_grip = [121.80, -17.03, 44.30, -83.75, 53.95, -39.95]
TCP_plate_red_1x3_grip2 = [116.78, -13.00, 56.07, -104.73, 53.86, -39.84]
TCP_plate_green_1x4_grip = [115.70, -32.98, 66.06, -100.65, 54.16, -30.07]
TCP_plate_blue_1x4_grip = [102.13, -51.38, 83.33, -112.62, 44.89, -12.56]

TCP_plate_2x2_grip1 = [117.93, -60.69, 102.88, -104.89, 51.40, 50.12] # drop
TCP_plate_2x2_grip1_1 = [110.74, -54.77, 115.02, -126.43, 50.65, 50.42]
TCP_plate_2x2_grip2 = [109.42, -60.13, 118.84, -108.74, 50.85, 51.17] # gripper close + up
TCP_plate_2x2_grip3 = [123.71, -48.32, 96.10, -106.46, 56.95, 41.49] # left side
TCP_plate_2x2_grip4 = [122.84, -50.20, 96.06, -117.05, 63.92, 45.48] # from left side -> push
TCP_plate_2x2_grip5 = [124.01, -70.08, 120.39, -112.52, 52.95, 132.66] # upper side
TCP_plate_2x2_grip6 = [121.18, -70.32, 110.82, -93.05, 61.81, 130.35] # upper side
##############################################################################
TCP_plate_2x2_grip7 = [114.72, -60.07, 106.57, -113.43, 52.72, -33.65] # stacking
# gripper open
TCP_plate_2x2_grip8 = [109.00, -60.55, 115.60, -123.96, 50.30, 150.01]
# gripper close
TCP_plate_2x2_grip9 = [113.03, -60.50, 107.55, -115.39, 50.14, 149.85]



rtde_c = rtde_control.RTDEControlInterface(host)

time.sleep(1)
gripper = RobotiqGripper(rtde_c)
gripper.activate()
gripper.set_speed(100)
gripper.set_force(100)
gripper.open()
time.sleep(3)
#
# rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
# 

#####################################
### Activate function
#####################################
start_time = time.time()

aa = 1
cc = 1
status = 0
x22 = 0

cc1 = ConveryorControl("192.168.10.122")
cc1.set_acceleration(50)
cc1.set_velocity(50)
cc1.move_continuous(1)

final = 0

while (aa != 0):
    
    ###########
    ### Move conveyor
    ###########
    while (cc != 0):
        cc1.move_continuous(1)
        temp = []
        
        if cc1.sensor() == 1: # detect object
            cc1.stop_continuous()
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
            time.sleep(2)
            bb = 1 # Activate camera
            final = final + 1
            break
    
    ##########
    ### Camera detect 
    ##########
    
    while (bb != 0):
        
        img = np.zeros((480,640,3))
        resp = requests.get("http://"+ "192.168.10.83"+":4242/current.jpg?type=color", stream=True).raw
        img = np.asarray(bytearray(resp.read()), dtype="uint8")
        img = cv2.imdecode(img, cv2.IMREAD_COLOR)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
        lb_red = np.array([150, 50, 70], "uint8")
        ub_red = np.array([200, 255, 255], "uint8")
        red_mask = cv2.inRange(hsv, lb_red, ub_red)

        lb_green = np.array([40, 150, 50], "uint8")
        ub_green = np.array([70, 255, 100], "uint8")
        green_mask = cv2.inRange(hsv, lb_green, ub_green)

        lb_blue = np.array([100, 200, 50], "uint8")
        ub_blue = np.array([130, 255, 255], "uint8")
        blue_mask = cv2.inRange(hsv, lb_blue, ub_blue)

        lb_yellow = np.array([0, 100, 100], "uint8")
        ub_yellow = np.array([50, 255, 255], "uint8")
        yellow_mask = cv2.inRange(hsv, lb_yellow, ub_yellow)

        kernel = np.ones((5, 5), "uint8")

        red_mask = cv2.dilate(red_mask, kernel)
        green_mask = cv2.dilate(green_mask, kernel)
        blue_mask = cv2.dilate(blue_mask, kernel)
        yellow_mask = cv2.dilate(yellow_mask, kernel)

        ########################
        ### RGB/Area of interests
        ########################
        min_area_red = 5000
        max_area_red = 20000

        min_area_green =  5000
        max_area_green =  20000

        min_area_blue =  3000
        max_area_blue =   20000

        min_area_yellow =  5000
        max_area_yellow =   20000

        area_x_min = 200
        area_x_max = 350
        area_y_min = 150
        area_y_max = 500

        ########################
        ### Red
        ########################
        contours, hierarchy = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if (min_area_red <= area and max_area_red >= area):
                x,y,w,h = cv2.boundingRect(contour)
                if (area_x_min <= x and x <= area_x_max):
                    if (area_y_min <= y and y <= area_y_max):
                        if w <= 80:
                            temp.append("red_1x3")
                        if w > 95:
                            temp.append("red_2x2")
                            

        ########################
        ### Green
        ########################
        contours, hierarchy = cv2.findContours(green_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if (min_area_green <= area and max_area_green >= area):
                x,y,w,h = cv2.boundingRect(contour)
                if (area_x_min <= x and x <= area_x_max):
                    if (area_y_min <= y and y <= area_y_max):
                        if h <= 180:
                            temp.append("green_1x2")
                        if h > 180:
                            temp.append("green_1x4")

        ########################
        ### Blue
        ########################
        contours, hierarchy = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if (min_area_blue <= area and max_area_blue >= area):
                x,y,w,h = cv2.boundingRect(contour)
                if (area_x_min <= x and x <= area_x_max):
                    if (area_y_min <= y and y <= area_y_max):
                        if h <= 100:
                            temp.append("blue_1x1")
                        if h > 100:
                            temp.append("blue_1x4")

        ########################
        ### Yellow
        ########################
        contours, hierarchy = cv2.findContours(yellow_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if area >= min_area_yellow and area <= max_area_yellow:
                x,y,w,h = cv2.boundingRect(contour)
                if (area_x_min <= x and x <= area_x_max):
                    if (area_y_min <= y and y <= area_y_max):
                        temp.append("yellow_2x2")
        
        if len(temp) >= 10:
            temp_color, temp_number = np.unique(np.array(temp), return_counts = True)
            if np.max(temp_number) >= 5:
                status = temp_color[np.argmax(temp_number)]
                bb = 0
    ###########
    ### Stacking
    ###########
    
    if status in ["red_2x2", "yellow_2x2"]:
        if x22 == 0:
            print(status)
            
            if status == "red_2x2":
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_red_1])
                time.sleep(0.5)
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_red_2])
                time.sleep(0.5)
                gripper.close()
                time.sleep(1)
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_red_1])
                time.sleep(0.5)
                
            if status == "yellow_2x2":
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_yellow_1])
                time.sleep(0.5)
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_yellow_2])
                time.sleep(0.5)
                gripper.close()
                time.sleep(1)
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_yellow_1])
                time.sleep(0.5)
            
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip1])
            time.sleep(0.5)
            gripper.open()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip1_1])
            time.sleep(0.5)
            gripper.close()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip2])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip3])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip4])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip3])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip5])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip6])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip5])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
            time.sleep(0.5)
            gripper.open()
            time.sleep(1)
            x22 = 1
            status = 0
            cc = 1
            
    if status in ["red_2x2", "yellow_2x2"]:      
        if x22 == 1:
            print(status)
            if status == "red_2x2":
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_red_1])
                time.sleep(0.5)
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_red_2])
                time.sleep(0.5)
                gripper.close()
                time.sleep(1)
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_red_1])
                time.sleep(0.5)
                
            if status == "yellow_2x2":
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_yellow_1])
                time.sleep(0.5)
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_yellow_2])
                time.sleep(0.5)
                gripper.close()
                time.sleep(1)
                rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_2x2_grip_yellow_1])
                time.sleep(0.5)
            
            rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip7])
            gripper.open()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip8])
            time.sleep(0.5)
            gripper.close()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip9])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_2x2_grip8])
            time.sleep(0.5)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
            time.sleep(0.5)
            gripper.open()
            time.sleep(1)
            status = 0
            cc = 1
    
    if status in ["red_1x3", "blue_1x4", "green_1x4"]:
        print(status)
        
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
        time.sleep(0.5)
        
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_1x_grip])    
        time.sleep(0.5)
        gripper.close()
        time.sleep(1)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_before_grip])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_conveyor_camera])
        time.sleep(0.5)
        rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
        time.sleep(0.5)
        
        if status == "red_1x3":
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_red_1x3_grip])
            gripper.open()
            time.sleep(1)
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_red_1x3_grip2])
            time.sleep(0.5)
            
        if status == "blue_1x4":
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_blue_1x4_grip])
            gripper.open()
            time.sleep(1)
        
        if status == "green_1x4":
            rtde_c.moveJ([a*3.1415/180 for a in TCP_plate_green_1x4_grip])
            gripper.open()
            time.sleep(1)
            
        rtde_c.moveJ([a*3.1415/180 for a in TCP_starting_point])
        time.sleep(0.5)
        status = 0
        cc = 1
    
    if final == 6:
        cc = 0
        
        cc1.stop_continuous()
        time.sleep(0.5)
        bb = 0
        time.sleep(0.5)
        aa = 0
        time.sleep(0.5)
        break
        
    

red_2x2
green_1x4
yellow_2x2
blue_1x4
blue_1x4
red_1x3


In [2]:
rtde_c.disconnect()